In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [14]:
# Check if GPU is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cpu


In [15]:

# Define Paths
training_path = "C:\\Users\\Victus\\Desktop\\Oragan Clasifier\\training"
val_path = "C:\\Users\\Victus\\Desktop\\Oragan Clasifier\\val"


In [16]:
# Define Transforms for Data Augmentation and Normalization
transform = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.RandomResizedCrop(224),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

In [17]:
# Load Data
train_data = datasets.ImageFolder(root=training_path, transform=transform['train'])
val_data = datasets.ImageFolder(root=val_path, transform=transform['val'])

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)


In [18]:

# Verify Class Labels
print("Class Labels:", train_data.classes)  # Should output: ['brain', 'hands', 'knee', 'lungs']


Class Labels: ['brain', 'hands', 'knee', 'lungs']


In [19]:

# Use Pre-trained Model (Transfer Learning) - ResNet18
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))  # Modify the output layer to 4 classes
model = model.to(device)

In [20]:

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [21]:
# Train the Model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50):
    best_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward Pass and Optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        # Evaluate on Validation Set
        val_accuracy = evaluate_model(model, val_loader)

        # Save the model if it achieves the best accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'organ_classifier.pth')
            print("Model saved!")

    print(f'Best Validation Accuracy: {best_accuracy:.4f}%')


In [22]:
def evaluate_model(model, loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')
    return accuracy

# Train the Model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)


Epoch [1/50], Loss: 0.5407
Validation Accuracy: 22.03%
Model saved!
Epoch [2/50], Loss: 0.3296
Validation Accuracy: 84.75%
Model saved!
Epoch [3/50], Loss: 0.2992
Validation Accuracy: 20.34%
Epoch [4/50], Loss: 0.2715
Validation Accuracy: 38.98%
Epoch [5/50], Loss: 0.1876
Validation Accuracy: 81.36%
Epoch [6/50], Loss: 0.0930
Validation Accuracy: 94.92%
Model saved!
Epoch [7/50], Loss: 0.3158
Validation Accuracy: 89.83%
Epoch [8/50], Loss: 0.2159
Validation Accuracy: 64.41%
Epoch [9/50], Loss: 0.1763
Validation Accuracy: 69.49%
Epoch [10/50], Loss: 0.0986
Validation Accuracy: 91.53%
Epoch [11/50], Loss: 0.2016
Validation Accuracy: 89.83%
Epoch [12/50], Loss: 0.1107
Validation Accuracy: 96.61%
Model saved!
Epoch [13/50], Loss: 0.0612
Validation Accuracy: 93.22%
Epoch [14/50], Loss: 0.0769
Validation Accuracy: 98.31%
Model saved!
Epoch [15/50], Loss: 0.1128
Validation Accuracy: 89.83%
Epoch [16/50], Loss: 0.0714
Validation Accuracy: 84.75%
Epoch [17/50], Loss: 0.0859
Validation Accuracy: